This code looks at the way in which the connections may or may not appear. 

In [2]:
import math
import pandas as pd
from midiutil import MIDIFile

MIDI defaults

In [16]:
track    = 0
channel  = 0
time     = 0    # In beats
duration = 1    # In beats
tempo    = 600   # In BPM
volume   = 62  # 0-127, as per the MIDI standard
# One track, defaults to format 1 (tempo track is created automatically)
# set tracks to max number of streams connected
MyMIDI = MIDIFile(4)  
MyMIDI.addTempo(track, time, tempo)

In [4]:
file_name='musicmile/bluetooth_1634469751686.txt'
bluetooth = pd.read_csv(file_name, header=None)
bluetooth.head()

,0,1,2,3,4,5,6,7,8,9
0,1634469751967,88:C6:26:D1:73:83,-52,1,0,0,,{3=[B@4c5bd9b},-2147483648,6
1,1634469751985,47:11:B1:77:DE:0C,-59,1,0,0,null,{76=[B@2c00a76},12,26
2,1634469751998,28:DD:73:41:DB:D7,-53,1,0,0,null,{},-2147483648,-1
3,1634469752020,17:77:17:99:A5:71,-49,1,0,0,null,{19456=[B@e944513},-2147483648,4
4,1634469752046,88:C6:26:D1:73:83,-37,1,0,0,,{3=[B@2d9f24e},-2147483648,6


In [5]:
start = bluetooth[0].min()
start

1634469751967

In [13]:
def clean_organisation(org):
    return org.replace('{', '').replace('}','').split('=')[0]

#extract the data parts. need a 
def clean_organisation_data(org):
    if len(org) > 3:
        return org.replace('{', '').replace('}','').split('=')[1]
    
    return 0

bluetooth['company'] = bluetooth[7].apply(clean_organisation)
bluetooth['data'] = bluetooth[7].apply(clean_organisation_data)
bluetooth

,0,1,2,3,4,5,6,7,8,9,company,data
0,1634469751967,88:C6:26:D1:73:83,-52,1,0,0,,{3=[B@4c5bd9b},-2147483648,6,3,[B@4c5bd9b
1,1634469751985,47:11:B1:77:DE:0C,-59,1,0,0,null,{76=[B@2c00a76},12,26,76,[B@2c00a76
2,1634469751998,28:DD:73:41:DB:D7,-53,1,0,0,null,{},-2147483648,-1,,0
3,1634469752020,17:77:17:99:A5:71,-49,1,0,0,null,{19456=[B@e944513},-2147483648,4,19456,[B@e944513
4,1634469752046,88:C6:26:D1:73:83,-37,1,0,0,,{3=[B@2d9f24e},-2147483648,6,3,[B@2d9f24e
...,...,...,...,...,...,...,...,...,...,...,...,...
9124,1634472712764,4A:80:17:1B:9A:16,-65,1,0,0,null,{76=[B@ce051b1},12,26,76,[B@ce051b1
9125,1634472712772,74:B6:B6:E6:08:82,-74,1,0,0,3.5.0-0-BAEAAvv/ug==-JABY0AWV,{},-2147483648,5,,0
9126,1634472712813,74:B6:B6:E6:25:42,-53,1,0,0,3.5.0-0-AQEABfv/qg==-JAC108FJ,{},-2147483648,5,,0
9127,1634472712836,62:A4:9E:6B:F1:A3,-56,1,0,0,null,{76=[B@e305e70},7,26,76,[B@e305e70


In [14]:
#time is in milliseconds from server. 
def add_rhythms(time, data_field):
    beat_time = math.floor(time - start)/100

    if data_field == 0:
        MyMIDI.addNote(track, channel, 35 ,beat_time, 0.5, 60)
    else:
        MyMIDI.addNote(track, channel, 48 ,beat_time, 0.3, 60)
        MyMIDI.addNote(track, channel, 50 ,beat_time + 0.1, 0.3, 60)

In [17]:
MyMIDI = MIDIFile(4)  
MyMIDI.addTempo(track, time, tempo)

def get_data(datum):
    print(datum)
    return [datum[0], datum[10]] 

new_rhythms = [get_data(blue) for blue in bluetooth.to_numpy()]
for data in new_rhythms:
    add_rhythms(data[0], data[1])

with open(file_name[:-4]+"connection.mid", "wb") as output_file:
    MyMIDI.writeFile(output_file)

[1634469751967 ' 88:C6:26:D1:73:83' -52 1 0 0 ' ' ' {3=[B@4c5bd9b}'
 -2147483648 6 ' 3' '[B@4c5bd9b']
[1634469751985 ' 47:11:B1:77:DE:0C' -59 1 0 0 ' null' ' {76=[B@2c00a76}'
 12 26 ' 76' '[B@2c00a76']
[1634469751998 ' 28:DD:73:41:DB:D7' -53 1 0 0 ' null' ' {}' -2147483648 -1
 ' ' 0]
[1634469752020 ' 17:77:17:99:A5:71' -49 1 0 0 ' null'
 ' {19456=[B@e944513}' -2147483648 4 ' 19456' '[B@e944513']
[1634469752046 ' 88:C6:26:D1:73:83' -37 1 0 0 ' ' ' {3=[B@2d9f24e}'
 -2147483648 6 ' 3' '[B@2d9f24e']
[1634469752067 ' 68:E5:47:7D:E0:04' -91 1 0 0 ' null' ' {76=[B@923c105}'
 12 26 ' 76' '[B@923c105']
[1634469755359 ' 17:77:17:99:A5:71' -79 1 0 0 ' null'
 ' {19456=[B@49dfd26}' -2147483648 4 ' 19456' '[B@49dfd26']
[1634469755368 ' DB:CA:C3:61:F7:6F' -47 1 0 0 ' null' ' {76=[B@c2e0bbd}'
 -2147483648 -1 ' 76' '[B@c2e0bbd']
[1634469755379 ' 44:91:8C:DB:9F:F7' -90 1 0 0 ' null' ' {76=[B@f7a4080}'
 26 26 ' 76' '[B@f7a4080']
[1634469755382 ' DE:C8:7F:80:A4:4E' -88 1 0 0 ' null' ' {76=[B@e51c25f}'
 -2

[1634470039996 ' 7C:00:98:D9:9C:8F' -87 1 0 0 ' null' ' {76=[B@fab1698}' 7
 26 ' 76' '[B@fab1698']
[1634470040013 ' 76:C7:E8:CB:BA:67' -87 1 0 0 ' null' ' {76=[B@7cac157}' 8
 26 ' 76' '[B@7cac157']
[1634470040032 ' D8:62:E8:14:D4:CE' -93 1 0 0 ' null' ' {76=[B@6d5be62}'
 -2147483648 -1 ' 76' '[B@6d5be62']
[1634470040046 ' 04:52:C7:AF:53:C8' -77 1 0 0 ' Welcome To Bubble Boba'
 ' {272=[B@9cdbb29}' 10 26 ' 272' '[B@9cdbb29']
[1634470040060 ' 88:C6:26:D1:73:83' -69 1 0 0 ' ' ' {3=[B@df097dc}'
 -2147483648 6 ' 3' '[B@df097dc']
[1634470040064 ' 66:00:98:BA:05:23' -83 1 0 0 ' null' ' {76=[B@811166b}'
 12 26 ' 76' '[B@811166b']
[1634470040068 ' 67:C3:74:71:B8:24' -88 1 0 0 ' null' ' {76=[B@9ce9586}' 8
 26 ' 76' '[B@9ce9586']
[1634470040075 ' 44:47:18:3F:53:1B' -78 1 0 0 ' null' ' {76=[B@e50479d}' 7
 26 ' 76' '[B@e50479d']
[1634470040086 ' 4A:5A:98:A8:8F:19' -78 1 0 0 ' null' ' {76=[B@20715e0}' 8
 26 ' 76' '[B@20715e0']
[1634470040097 ' 6A:54:FD:F8:03:A9' -85 1 0 0 ' null' ' {76=[B@8771f3f}' 8

 -2147483648 26 ' 76' '[B@345ad74']
[1634470290634 ' 44:93:41:C9:52:86' -91 1 0 0 ' null' ' {76=[B@44cfae3}' 7
 26 ' 76' '[B@44cfae3']
[1634470290639 ' 06:A7:69:50:EF:25' -91 1 0 0 ' null' ' {}' -2147483648 26
 ' ' 0]
[1634470290669 ' 04:90:1B:8F:35:EF' -94 1 0 0 ' null' ' {76=[B@455370c}'
 -2147483648 26 ' 76' '[B@455370c']
[1634470290691 ' DB:CA:C3:61:F7:6F' -48 1 0 0 ' null' ' {76=[B@c8ce35b}'
 -2147483648 -1 ' 76' '[B@c8ce35b']
[1634470290714 ' 00:00:00:00:00:00' -85 1 0 0 ' null'
 ' {51215=[B@106bb36}' -2147483648 -1 ' 51215' '[B@106bb36']
[1634470290718 ' 2F:8E:6A:E0:06:76' -81 1 0 0 ' null' ' {}' -2147483648 26
 ' ' 0]
[1634470290737 ' 64:1C:B0:9B:C5:54' -92 1 0 0 ' null' ' {117=[B@fe262d3}'
 -2147483648 -1 ' 117' '[B@fe262d3']
[1634470290754 ' 00:00:00:00:00:00' -85 1 0 0 ' null'
 ' {26735=[B@3adfb0e}' -2147483648 -1 ' 26735' '[B@3adfb0e']
[1634470290770 ' 76:B3:C8:79:FB:79' -48 1 0 0 ' null' ' {76=[B@b3af0c5}' 7
 26 ' 76' '[B@b3af0c5']
[1634470290773 ' 6E:83:99:81:71:06' -89 1

 ' ' 0]
[1634470688123 ' BE:58:60:02:99:4B' -82 1 0 0 ' ELK-BLEDOM   ' ' {}'
 -2147483648 6 ' ' 0]
[1634470688130 ' 6F:83:F1:36:72:AC' -57 1 0 0 ' null' ' {76=[B@8572ac1}' 7
 26 ' 76' '[B@8572ac1']
[1634470688142 ' 6D:21:DD:9F:69:81' -86 1 0 0 ' null' ' {}' -2147483648 2
 ' ' 0]
[1634470688161 ' 6A:AD:08:38:C1:1F' -85 1 0 0
 ' Galaxy Watch4 Classic (7EWN)' ' {117=[B@cd70ef2}' -2147483648 2 ' 117'
 '[B@cd70ef2']
[1634470699725 ' 7E:25:01:21:FC:FD' -86 1 0 0 ' null' ' {76=[B@6d323b5}'
 12 26 ' 76' '[B@6d323b5']
[1634470699745 ' 88:C6:26:D1:73:83' -64 1 0 0 ' ' ' {3=[B@7e652d8}'
 -2147483648 6 ' 3' '[B@7e652d8']
[1634470699814 ' 72:6C:8C:31:AB:44' -95 1 0 0 ' null' ' {76=[B@52b5a97}' 7
 26 ' 76' '[B@52b5a97']
[1634470699819 ' 54:AB:50:B2:98:8C' -71 1 0 0 ' null' ' {76=[B@ed18a2}' 12
 26 ' 76' '[B@ed18a2']
[1634470699821 ' 7B:2C:D5:2F:18:40' -70 1 0 0 ' null' ' {76=[B@3d78a69}'
 12 26 ' 76' '[B@3d78a69']
[1634470699830 ' 88:C6:26:D1:73:83' -59 1 0 0 ' ' ' {3=[B@d53201c}'
 -2147483648 6 ' 3

 -2147483648 6 ' 3' '[B@1db05bd']
[1634471143694 ' 04:E4:76:9B:37:51' -70 1 0 0 ' null' ' {}' -2147483648 -1
 ' ' 0]
[1634471143724 ' 88:C6:26:D1:73:83' -73 1 0 0 ' ' ' {3=[B@3551cfe}'
 -2147483648 6 ' 3' '[B@3551cfe']
[1634471143727 ' 88:C6:26:D1:73:83' -70 1 0 0 ' ' ' {3=[B@b822f75}'
 -2147483648 6 ' 3' '[B@b822f75']
[1634471143811 ' 88:C6:26:D1:73:83' -74 1 0 0 ' ' ' {3=[B@10aa198}'
 -2147483648 6 ' 3' '[B@10aa198']
[1634471143826 ' 6D:68:4F:42:C7:62' -72 1 0 0 ' null' ' {76=[B@96c0857}'
 -2147483648 26 ' 76' '[B@96c0857']
[1634471143838 ' 7E:97:22:EE:47:E9' -79 1 0 0 ' null' ' {76=[B@900f162}'
 12 26 ' 76' '[B@900f162']
[1634471143849 ' 54:AB:50:B2:98:8C' -73 1 0 0 ' null' ' {76=[B@2ccca29}'
 12 26 ' 76' '[B@2ccca29']
[1634471143895 ' 88:C6:26:D1:73:83' -71 1 0 0 ' ' ' {3=[B@c4a32dc}'
 -2147483648 6 ' 3' '[B@c4a32dc']
[1634471143973 ' 04:E4:76:9B:37:51' -79 1 0 0 ' null' ' {}' -2147483648 -1
 ' ' 0]
[1634471143976 ' 88:C6:26:D1:73:83' -70 1 0 0 ' ' ' {3=[B@a029561}'
 -2147483648 6 

 ' ' 0]
[1634471660069 ' 88:C6:26:D1:73:83' -51 1 0 0 ' ' ' {3=[B@516e07a}'
 -2147483648 6 ' 3' '[B@516e07a']
[1634471660149 ' 88:C6:26:D1:73:83' -62 1 0 0 ' ' ' {3=[B@89c7621}'
 -2147483648 6 ' 3' '[B@89c7621']
[1634471660208 ' 7E:97:22:EE:47:E9' -67 1 0 0 ' null' ' {76=[B@280e934}'
 12 26 ' 76' '[B@280e934']
[1634471660228 ' 64:E7:D8:8A:A2:91' -89 1 0 0 ' null' ' {117=[B@5cf39a3}'
 -2147483648 -1 ' 117' '[B@5cf39a3']
[1634471660235 ' 88:C6:26:D1:73:83' -64 1 0 0 ' ' ' {3=[B@5729c1e}'
 -2147483648 6 ' 3' '[B@5729c1e']
[1634471660248 ' 6F:83:F1:36:72:AC' -51 1 0 0 ' null' ' {76=[B@6589115}' 7
 26 ' 76' '[B@6589115']
[1634471660287 ' 04:E4:76:9B:37:51' -51 1 0 0 ' null' ' {}' -2147483648 -1
 ' ' 0]
[1634471660303 ' 6D:68:4F:42:C7:62' -67 1 0 0 ' null' ' {76=[B@ec810f6}'
 12 26 ' 76' '[B@ec810f6']
[1634471660346 ' 17:7A:B3:40:A8:70' -94 1 0 0 ' null' ' {}' -2147483648 -1
 ' ' 0]
[1634471671849 ' 64:E7:D8:8A:A2:91' -78 1 0 0 ' null' ' {117=[B@2bdf3ef}'
 -2147483648 -1 ' 117' '[B@2bdf3ef']

 -2147483648 -1 ' 6' '[B@235332']
[1634472041947 ' 84:C0:EF:CC:37:44' -79 1 0 0
 ' [TV] Samsung 6 Series (49)' ' {117=[B@c094a00}' -2147483648 26 ' 117'
 '[B@c094a00']
[1634472041949 ' 41:35:A6:A6:F0:B8' -50 1 0 0 ' null' ' {76=[B@ded79df}'
 -2147483648 26 ' 76' '[B@ded79df']
[1634472041987 ' 41:35:A6:A6:F0:B8' -50 1 0 0 ' null' ' {76=[B@74e168a}'
 -2147483648 26 ' 76' '[B@74e168a']
[1634472042013 ' 41:35:A6:A6:F0:B8' -50 1 0 0 ' null' ' {76=[B@b296f71}'
 -2147483648 26 ' 76' '[B@b296f71']
[1634472042021 ' 88:C6:26:D1:73:83' -65 1 0 0 ' ' ' {3=[B@353eec4}'
 -2147483648 6 ' 3' '[B@353eec4']
[1634472042026 ' 84:C0:EF:CC:37:44' -79 1 0 0
 ' [TV] Samsung 6 Series (49)' ' {117=[B@a67e873}' -2147483648 26 ' 117'
 '[B@a67e873']
[1634472042040 ' 41:35:A6:A6:F0:B8' -43 1 0 0 ' null' ' {76=[B@156f52e}'
 -2147483648 26 ' 76' '[B@156f52e']
[1634472042042 ' 3B:D8:65:41:B7:3A' -91 1 0 0 ' null' ' {6=[B@d034965}'
 -2147483648 -1 ' 6' '[B@d034965']
[1634472042103 ' 41:35:A6:A6:F0:B8' -55 1 0 0 ' null'

 12 26 ' 76' '[B@9485f7e']
[1634472336318 ' 88:C6:26:D1:73:83' -67 1 0 0 ' ' ' {3=[B@dd633f5}'
 -2147483648 6 ' 3' '[B@dd633f5']
[1634472336324 ' 49:CE:3C:D9:F1:ED' -85 1 0 0 ' null' ' {76=[B@c3fd018}'
 12 26 ' 76' '[B@c3fd018']
[1634472336326 ' 74:74:ED:96:F0:0B' -85 1 0 0 ' null' ' {76=[B@8abe8d7}'
 24 26 ' 76' '[B@8abe8d7']
[1634472336360 ' 41:35:A6:A6:F0:B8' -56 1 0 0 ' null' ' {76=[B@888abe2}'
 12 26 ' 76' '[B@888abe2']
[1634472351956 ' 32:38:8E:A7:38:FC' -51 1 0 0 ' null' ' {}' -2147483648 -1
 ' ' 0]
[1634472351972 ' 88:C6:26:D1:73:83' -57 1 0 0 ' ' ' {3=[B@e9e81e1}'
 -2147483648 6 ' 3' '[B@e9e81e1']
[1634472352008 ' 74:74:ED:96:F0:0B' -81 1 0 0 ' null' ' {76=[B@f5437f4}'
 24 26 ' 76' '[B@f5437f4']
[1634472352012 ' 3C:03:8B:50:3F:F1' -93 1 0 0 ' null' ' {}' -2147483648 26
 ' ' 0]
[1634472352018 ' 4A:80:17:1B:9A:16' -70 1 0 0 ' null' ' {76=[B@ec8c19}' 12
 26 ' 76' '[B@ec8c19']
[1634472352033 ' 26:D0:B9:9F:EB:E2' -85 1 0 0 ' null' ' {}' -2147483648 -1
 ' ' 0]
[1634472352068 ' 74:B6